In [23]:
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import numpy as np
import pandas as pd
from __future__ import division
from math import sqrt
import os
import fnmatch
import glob
import csv
import statistics
import calendar
from datetime import datetime
from sklearn import metrics

In [24]:
def convertToUnix(timelist):
    datetime_list=[]
    for t in range(len(timelist)):
        datetime_object = datetime.strptime(timelist[t], '%H:%M:%S %s')
        timestamp=calendar.timegm(datetime_object.utctimetuple())
        datetime_list.append(timestamp)
    return datetime_list

In [25]:
#calculate BW (0.707*Max)
def BwTime(timelist,cgmlist,maxvalue,MaxIndex):
    threshold_val = 0.5*maxvalue
    size = len(cgmlist) 
    index = MaxIndex
    for d in range(size):
        if(index!=0):
            index-=1 
        if(index == 0):
            bandwidth_lower = index
            break           
        elif(cgmlist[index] < threshold_val):            
            bandwidth_lower = index
            break
    index = MaxIndex
    for d in range(size):
        index+=1
        if(index == size):
            bandwidth_upper = index
            break
        elif(cgmlist[index] < threshold_val):
            bandwidth_upper = index
            break
    bw_time = (bandwidth_upper - bandwidth_lower) * 5 #5 minutes per point       
    return bw_time

In [26]:
for csvfile in glob.glob(os.path.join('/home/sahulphaniraj/csce633-ml/MachineLearningProject/Dataset/PlosJournalData/Consolidated-features/', '*.csv')):
    RelTimelist = []
    filename = csvfile
    csv_input = pd.read_csv(filename)
    csv_input.columns = ['Timestamp','Glucose']
    for i in range(len(csv_input['Timestamp'])):
        time=i*300 #time in seconds       
        RelTimelist.append(time)
    csv_input["RelTime"] = RelTimelist    
    #Megha feature extraction - Mean, SD, AUC
    FeatureMean = csv_input['Glucose'].mean()
    Featurestd = statistics.stdev(csv_input['Glucose'])
    csv_input["FeatureMean"] = FeatureMean
    csv_input["FeatureStd"] = Featurestd
    datetime_list = csv_input['RelTime']
    glucose = csv_input['Glucose']
    FeatureAUC = sklearn.metrics.auc(datetime_list, glucose,reorder=True)
    csv_input["FeatureAUC"] = FeatureAUC
    #Sahul feature extraction - Peak, MaxtimeIndex, Half peak
    FeaturePeak = csv_input['Glucose'].max()
    csv_input["FeaturePeak"] = FeaturePeak    
    MaxIndex = csv_input['Glucose'].idxmax(0)
    csv_input["MaxIndex"] = MaxIndex
    bandwidth_list= BwTime(csv_input['RelTime'],csv_input['Glucose'],FeaturePeak,MaxIndex)
    csv_input["BandwidthTime"] = bandwidth_list
    #Projna feature extraction - skewness, last half an hour CGM mean
    #this calculates skewness
    FeatureSkew = csv_input['Glucose'].skew() 
    csv_input["FeatureSkew"] = FeatureSkew
    #this calculates mean from last half an hour data.
    t = len(csv_input['RelTime'])
    temp =[]
    for i in range(1, 8):
        temp.append(csv_input['Glucose'][t-i])
    temp= np.array(temp)
    FeatureLastHHMean = temp.mean()
    csv_input["FeatureLastHHMean"] = FeatureLastHHMean
    #write to file
    csv_input.to_csv(filename, index=False)
   
    

ValueError: Length mismatch: Expected axis has 11 elements, new values have 2 elements

In [27]:
csv_input

,Timestamp,Glucose,RelTime,FeatureMean,FeatureStd,FeatureAUC,FeaturePeak,MaxIndex,BandwidthTime,FeatureSkew,FeatureLastHHMean
0,11:30:00 AM,96,0,92.432432,11.83216,998100.0,114,17,185,0.014158,88.857143
1,11:35:00 AM,82,300,92.432432,11.83216,998100.0,114,17,185,0.014158,88.857143
2,11:40:00 AM,79,600,92.432432,11.83216,998100.0,114,17,185,0.014158,88.857143
3,11:45:00 AM,75,900,92.432432,11.83216,998100.0,114,17,185,0.014158,88.857143
4,11:50:00 AM,73,1200,92.432432,11.83216,998100.0,114,17,185,0.014158,88.857143
5,11:55:00 AM,75,1500,92.432432,11.83216,998100.0,114,17,185,0.014158,88.857143
6,12:00:00 PM,76,1800,92.432432,11.83216,998100.0,114,17,185,0.014158,88.857143
7,12:05:00 PM,74,2100,92.432432,11.83216,998100.0,114,17,185,0.014158,88.857143
8,12:10:00 PM,76,2400,92.432432,11.83216,998100.0,114,17,185,0.014158,88.857143
9,12:15:00 PM,79,2700,92.432432,11.83216,998100.0,114,17,185,0.014158,88.857143


In [20]:
a=csv_input['Timestamp'][0].split(" ")

In [21]:
a

['11:30:00', 'AM']

In [22]:
a[0]

'11:30:00'

In [5]:
#calculate BW (0.707*Max)
def BwTime(timelist,cgmlist,maxvalue,MaxIndex):
    threshold_val = 0.5*maxvalue
    size = len(cgmlist) 
    print(size)
    print(timelist)
    print(cgmlist)
    print(maxvalue)
    print(MaxIndex)
    index = MaxIndex
    for d in range(size):
        if(index!=0):
            index-=1 
        if(index == 0):
            bandwidth_lower = index
            break           
        elif(cgmlist[index] < threshold_val):            
            bandwidth_lower = index
            break
    index = MaxIndex
    for d in range(size):
        index+=1
        if(index == size):
            bandwidth_upper = index
            break
        elif(cgmlist[index] < threshold_val):
            bandwidth_upper = index
            break
    bw_time = (bandwidth_upper - bandwidth_lower) * 5 #5 minutes per point       
    return bw_time

In [23]:
#csv_input['Glucose']
#csv_input['timeinsec']
#FeaturePeak
#MaxIndex
bandwidth_list= BwTime(csv_input['timeinsec'],csv_input['Glucose'],FeaturePeak,MaxIndex)
bandwidth_list


38
0     1473852600
1     1473852900
2     1473852900
3     1473853200
4     1473853500
5     1473853800
6     1473854100
7     1473854400
8     1473854700
9     1473855000
10    1473855300
11    1473855600
12    1473855900
13    1473856200
14    1473856500
15    1473856800
16    1473857100
17    1473857400
18    1473857700
19    1473858000
20    1473858300
21    1473858600
22    1473858900
23    1473859200
24    1473859500
25    1473859800
26    1473860100
27    1473860400
28    1473860700
29    1473861000
30    1473861300
31    1473861600
32    1473861900
33    1473862200
34    1473862500
35    1473862800
36    1473863100
37    1473863400
Name: timeinsec, dtype: int64
0      96
1      82
2      80
3      79
4      75
5      73
6      75
7      76
8      74
9      76
10     79
11     82
12     91
13     97
14    103
15    107
16    110
17    113
18    114
19    111
20    107
21    106
22    100
23     97
24     96
25     97
26     95
27     96
28     97
29     97
30     97
31     92
3

190

In [ ]:
#calculate BW (0.707*Max)
def BwTime(timelist,cgmlist,maxvalue,MaxIndex):
    threshold_val = 0.707*maxvalue
    size = len(cgmlist) 
    index = MaxIndex
    for d in range(size):
        index-=1
        print(index)
        print(cgmlist[index])
        if(cgmlist[index] < threshold_val):
            bandwidth_lower = timelist[index]
            break
    index = MaxIndex
    for d in range(size):
        index+=1
        if(cgmlist[index] < threshold_val):
            bandwidth_upper = timelist[index]
            break
    bw_time = bandwidth_upper - bandwidth_lower       
    return bw_time

In [11]:
a=[1,2,3]
a.index(1)

0

In [2]:
# Run this file within the directory which has all the csv data files. 
#It reads all the csv files within the directory and treats them to be input
for csvfile in glob.glob(os.path.join('.', '*.csv')):
    filename = csvfile[2:]
    csv_input = pd.read_csv(filename)
    #print(filename)
    csv_input.columns = ['Timestamp','Glucose']
    FeatureMean = csv_input['Glucose'].mean()
    Featurestd = statistics.stdev(csv_input['Glucose'])
    csv_input["Feature mean"] = FeatureMean
    csv_input["Feature std"] = Featurestd
    datetime_list= convertToUnix(csv_input['Timestamp'])
    glucose = csv_input['Glucose']
    FeatureAUC = sklearn.metrics.auc(datetime_list, glucose,reorder=True)
    csv_input["Feature AUC"] = FeatureAUC
    csv_input.to_csv("features/"+filename, index=False)

In [3]:
def convertToUnix(timelist):
    datetime_list=[]
    for t in range(len(timelist)):
        #print(timelist[t])
        datetime_object = datetime.strptime(timelist[t], '%Y-%m-%d  %H:%M:%S')
        timestamp=calendar.timegm(datetime_object.utctimetuple())
        datetime_list.append(timestamp)
    return datetime_list